## FDW Crop Production Data Profiling - Senegal

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32628' # Senegal
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/SN_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Senegal',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1960	 | SN1960A1 | 7	| SN1960A2	| 28	| nan	| 0	|
| 1980	 | SN1980A1 | 8	| SN1980A2	| 28	| nan	| 0	|
| 1984	 | SN1984A1 | 10	| SN1984A2	| 30	| nan	| 0	|
| 2002	 | SN2002A1 | 11	| SN2002A2	| 34	| nan	| 0	|
| 2006	 | nan | 0	| SN2006A2	| 35	| nan	| 0	|
| 2008	 | SN2008A1 | 14	| SN2008A2	| 45	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual setting
# a) Add admin names to reporting units
admin_level = 'admin_2'
reporting_unit = df.loc[df['fnid'].map(lambda x: x[6] != 'A'), 'fnid'].unique()
df.loc[df.fnid.isin(reporting_unit),admin_level] = df.loc[df.fnid.isin(reporting_unit), 'locality_name'].map(lambda x: x.split(' ')[0])
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority='Area Planted')
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([
    SN_Admin2_1960, SN_Admin2_1980, SN_Admin2_1984,
    SN_Admin2_2002, SN_Admin2_2006, SN_Admin2_2008
], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 12,654
Removed 92 "Missing Value" points
4,214/4,218 "Area Planted" points are retained.
4,176/4,218 "Quantity Produced" points are retained.
4,172/4,218 "Yield" points are retained.
Current data points: 12,562

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1960 - 2015
5 grain types are found: Fonio, Maize (Corn), Millet (Bulrush), Rice (Paddy), Sorghum
2 seasons are found: Main season (12-01), Off season (02-01)
3 crop production system are found: none, irrigated, recessional (PS)
Data sources include:
[1] Agence Nationale de la Statistique et de la Demographie, Senegal --- Situation Socio-Economique, Senegal
[2] Agence Nationale de la Statistique, Senegal --- Situation Socio-Economique, Senegal
[3] Ministere de l'agriculture et du developpement rural, DAPSA, Senegal --- FEWS NET Agro Maps, Senegal
[4] Ministere de l'agriculture et du developpemen

- Senegal crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-senegal.png?itok=wS-VwgSv)

- FDW data consists of `SN1960A2`, `SN1980A2`, `SN1984A2`, `SN2002A2`, `SN2006A2`, and `SN2008A2`.

| Year | Admin-1 | # units  | Admin-2  | # units |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1960	 | SN1960A1 | 7	| **`SN1960A2`**	| 28	|
| 1980	 | SN1980A1 | 8	| **`SN1980A2`**	| 28	|
| 1984	 | SN1984A1 | 10	| **`SN1984A2`**	| 30	|
| 2002	 | SN2002A1 | 11	| **`SN2002A2`**	| 34	|
| 2006	 | - | -	| **`SN2006A2`**	| 35	|
| 2008	 | SN2008A1 | 14	| **`SN2008A2`**	| 45	|

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SN_admin_shapes.png?raw=true)

- In 1980, no district is divided and added.

- In 1984, 2 districts are divided and added.

| 1980-1984 | 1984-2002|
| :---:|:---:|
|SN1980A20101 (Dakar) | SN1984A20102 (Pikine), SN1984A20103 (Rufisque), SN1984A20104 (Dakar)|

- In 2002, 4 districts are divided and added.
| 1984-2002 | 2002-2006|
| :---:|:---:|
|SN1984A20102 (Pikine) | SN2002A20105 (Pikine), SN2002A20106 (Guediawaye)|
|SN1984A20401 (Dagana) | SN2002A21701 (Dagana), SN2002A21703 (Saint-Louis)|
|SN1984A20402 (Matam) | SN2002A21801 (Kanel), SN2002A21802 (Matam), SN2002A21803 (Ranerou Ferlo)|

- In 2006, 1 district is divided and added.
| 2002-2006 | 2006-2008 |
| :---:|:---:|
|SN2002A21601 (Kaffrine) | SN2006A21601 (Kaffrine), SN2006A21604 (Koungheul)|

- In 2008, 10 district are divided and added.
| 2006-2008 | 2008-present |
| :---:|:---:|
|SN2006A21503 (Gossas) | SN2008A22401 (Guinguineo), SN2008A22603 (Gossas)|
|SN2006A20502 (Kedougou) | SN2008A22301 (Kedougou), SN2008A22302 (Salemata), SN2008A22303 (Kolda)|
|SN2006A21101 (Kolda) | SN2008A22001 (Kolda), SN2008A22003 (Medina Yoro Foulah)|
|SN2006A21102 (Sedhiou) | SN2008A22101 (Bounkiling), SN2008A22102 (Goudomp), SN2008A22103 (Sedhiou)|
|SN2006A20501 (Bakel) | SN2008A22201 (Bakel), SN2008A22202 (Goudiry)|
|SN2006A20503 (Tambacounda) | SN2008A22203 (Koumpentoum), SN2008A22204 (Tambacounda)|
|SN2006A21601 (Kaffrine) | SN2008A22501 (Birkelane), SN2008A22502 (Kaffrine), SN2008A22504 (Malem Hoddar)|

- **`SN2008A2`** is used to represent current admin-level 2 crop data.
- Senegal has 2 crop seasons: Main season (12-01), Off season (02-01).
- Senegal has 3 crop production systems: None, irrigated, recessional (PS).
- Senegal has no population group(s).

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = SN_Admin2_2008.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [5]:
# Link admin boundaries ------------------------- #
link_1960, over_1960 = FDW_PD_CreateAdminLink(SN_Admin2_1960, SN_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_1960['SN1960A20101'].update(method='PBR')
link_1960['SN1960A20304'].update(method='PBR')
link_1960['SN1960A20402'].update(method='PBR')
assert all(np.unique([v['method'] for k,v in link_1960.items()]) == 'PBR')
link_1980, over_1980 = FDW_PD_CreateAdminLink(SN_Admin2_1980, SN_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_1980['SN1980A20101'].update(method='PBR')
link_1980['SN1980A20902'].update(method='PBR')
link_1980['SN1980A20402'].update(method='PBR')
assert all(np.unique([v['method'] for k,v in link_1980.items()]) == 'PBR')
link_1984, over_1984 = FDW_PD_CreateAdminLink(SN_Admin2_1984, SN_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_1984['SN1984A20104'].update(method='PBR')
link_1984['SN1984A20102'].update(method='PBR')
link_1984['SN1984A20902'].update(method='PBR')
link_1984['SN1984A20402'].update(method='PBR')
assert all(np.unique([v['method'] for k,v in link_1984.items()]) == 'PBR')
link_2002, over_2002 = FDW_PD_CreateAdminLink(SN_Admin2_2002, SN_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2002['SN2002A20104'].update(method='PBR')
link_2002['SN2002A20106'].update(method='PBR')
link_2002['SN2002A20105'].update(method='PBR')
assert all(np.unique([v['method'] for k,v in link_2002.items()]) == 'PBR')
link_2006, over_2006 = FDW_PD_CreateAdminLink(SN_Admin2_2006, SN_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2006['SN2006A20104'].update(method='PBR')
link_2006['SN2006A20106'].update(method='PBR')
link_2006['SN2006A20105'].update(method='PBR')
assert all(np.unique([v['method'] for k,v in link_2006.items()]) == 'PBR')
# Crop specific ratios
link_ratio_1960 = FDW_PD_RatioAdminLink(link_1960, prod, over_1960, mdx_pss)
link_ratio_1980 = FDW_PD_RatioAdminLink(link_1980, prod, over_1980, mdx_pss)
link_ratio_1984 = FDW_PD_RatioAdminLink(link_1984, prod, over_1984, mdx_pss)
link_ratio_2002 = FDW_PD_RatioAdminLink(link_2002, prod, over_2002, mdx_pss)
link_ratio_2006 = FDW_PD_RatioAdminLink(link_2006, prod, over_2006, mdx_pss)
# Merge link_ratio
assert link_ratio_1960.keys() == link_ratio_1980.keys()
assert link_ratio_1980.keys() == link_ratio_1984.keys()
assert link_ratio_1984.keys() == link_ratio_2002.keys()
assert link_ratio_2002.keys() == link_ratio_2006.keys()
link_merged = [link_ratio_1960, link_ratio_1980, link_ratio_1984, link_ratio_2002, link_ratio_2006]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

CBR is considered for 'SN1960A20101' as no record found in: ['SN2008A20104', 'SN2008A20105', 'SN2008A20106']
CBR is considered for 'SN1980A20101' as no record found in: ['SN2008A20104', 'SN2008A20105', 'SN2008A20106']
CBR is considered for 'SN1984A20104' as no record found in: ['SN2008A20104']
CBR is considered for 'SN1984A20102' as no record found in: ['SN2008A20105', 'SN2008A20106']
CBR is considered for 'SN2002A20104' as no record found in: ['SN2008A20104']
CBR is considered for 'SN2002A20106' as no record found in: ['SN2008A20106']
CBR is considered for 'SN2002A20105' as no record found in: ['SN2008A20105']
CBR is considered for 'SN2006A20104' as no record found in: ['SN2008A20104']
CBR is considered for 'SN2006A20106' as no record found in: ['SN2008A20106']
CBR is considered for 'SN2006A20105' as no record found in: ['SN2008A20105']


In [6]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack_gscd = stack[names]
stack_gscd['gscd_code'] = 'calibrated'
# ----------------------------------------------- #

# Reported FDW data ----------------------------- #
stack_fdw = df[names]
stack_fdw['indicator'] = stack_fdw['indicator'].replace({'Area Harvested':'area','Quantity Produced':'production','Yield':'yield'})
stack_fdw['gscd_code'] = 'reported'
# ----------------------------------------------- #

# Final Processing ------------------------------ #
stack = pd.concat([stack_fdw,stack_gscd], axis=0).reset_index(drop=True)
# No concerns found for grain types
stack['product'] = stack['product'].replace(product_category)
# Calibration of planting and Harvest year and season
cs = {
    'Main season': {'planting_month':{'12-01':'06-01'},'harvest_month':{'12-01':'11-01'}},
    'Off season': {'planting_month':{'02-01':'02-01'},'harvest_month':{'07-01':'07-01'}},
}
cy = {}
stack, df, link_ratio = FDW_PD_CaliSeasonYear(stack, df, link_ratio, cs, cy)
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = './data/crop/adm_crop_production_SN.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_hdf('./data/crop/adm_crop_production_SN.hdf', stack)
save_hdf('./data/crop/adm_crop_production_SN_raw.hdf', df)
save_npz('./data/crop/adm_crop_production_SN_ratio.npz', link_ratio)

./data/crop/adm_crop_production_SN.csv is saved.
./data/crop/adm_crop_production_SN.hdf is saved.
./data/crop/adm_crop_production_SN_raw.hdf is saved.
./data/crop/adm_crop_production_SN_ratio.npz is saved.


## Visualization of production data

In [7]:
# Bar chart of national grain production
country_iso, country_name = 'SN', 'Senegal'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Sorghum','Millet','Maize','Rice','Fonio']
for season_name in ['Main season']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = './figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()

./figures/SN_bar_natgrainprod_Main season.png is saved.


![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/SN_bar_natgrainprod_Main.png?raw=true)

In [8]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'SN', 'Senegal'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','Main season'],
    ['Sorghum','Main season'],
    ['Millet','Main season'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = './figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

./figures/SN_line_pay_Maize_Main season.png is saved.


./figures/SN_line_pay_Sorghum_Main season.png is saved.


./figures/SN_line_pay_Millet_Main season.png is saved.


<!-- ![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/ML_line_pay_Maize_Main.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/ML_line_pay_Millet_Main.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/ML_line_pay_Rice_Main.png?raw=true)
![image](https://github.com/chc-ucsb/GlobalCropData/blob/main/figures/ML_line_pay_Sorghum_Main.png?raw=true) -->

In [9]:
# # Calibrated PAY time-series per FNID
# from tools_graphic import PlotLineCropTS
# country_iso, country_name = 'ML', 'Mali'
# df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
# link_ratio = load_npz('./data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
# year_all = np.arange(df['year'].min(), df['year'].max()+1)
# prod_season = [
#     ['Maize', 'Main'],
#     ['Millet', 'Main']
# ]
# for product_name, season_name in prod_season:
#     sub = df[
#         (df['product'] == product_name) &
#         (df['season_name'] == season_name)
#     ]
#     for fnid in sub['fnid'].unique():
#         sub_fps = sub[sub['fnid'] == fnid]
#         fn_save = './figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
#         # fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)